# TechTogether 2020 -- IBM Researh Challenge

## Task 1


"Design and develop a web-based annotation tool that makes it easy to annotate scientific articles by a diverse crowd. "

**Response**

At the very beginning, around 9pm 2/1, we planned to integrate the AWS and their language comprehension APIs together with some custom processing programming as our on-the-fly backend. However, unfortunately, after 4 hours' trial and errors mentor and we had reached a consensus that we should give up the wrap and focus the "meant".

Therefore, we moved to use jupyter notebook as an alternative way to illustrate our thinking process and design consideration. Also, as an illustartive as well as interactive demo.


## Motivation of Word Filtering

The core demand is that "scaling the annotations to a more diverse crowd that do not necessarily have a solid background in biology", so we want to develop an way using the linguistic properties of texts (which is universal to all kinds of writing style and quite roubust and effective when using as a feature in text classification problem) and the type information of the name entities. Name entities are the common phrases when people refer to something/someone in their language.(Of course, the pre-trained medical name entitity recognizer will work and best and actually solve the problem, save we from working this annotation automation challenge, but I guess we are implicitly banned to use this kind of tool).We beleieve that, by filtering words of abstracts with their part of speech tag and name entitty tag e.g gene name, we can filter most non-target words and save our annotators more time to annotation more data entires.

The next question/step is to find some simple and easy-to-identify features that the cancer name and drug name exclusively have.Since our users are assumed to have zero background in medicine, nursing, biomedical, chemistry, etc. background, so we need to think in their shoes: what do they know about cancer and drug names? What do they don't know and/or have never been exposed to? Something very obvious to catch would be better (save theri effort and stackholders' human labor cost)!

We reasoned that,the frequency info of that noun in the common vocabulary is a very informative indicator -- the smaller the frequency is, the higher chance the word is going to be medical-related.But we don't have that statistics at hand while this corpus is both too small and too biased (not common) to be used for the statistic estimation. So we didn't implement this attribute. Another good indicator to use is the length of the noun but this is not always reliable, because medical vocabulary tends to have weirdly long letter length.So we'll provide as machine's silly tip when asking annotator to annotate, the final judgement will be left to them.

By far, we think we'll be able to make things much easier for the annotator, we assume they will easily to annotate words based on how familiar they are with the word,i.e. if they never see that word or don't know what it means, they probably should label it and it would be medical.

One thing worths to be mentioned is that, we assume **this annotation is carried out under the premise that the presenting abstracts are all already filtered based on their content**,that is data collectors have made sure that the paper discusses the desirable phenotyes of the cancers of interest.

### Prep code

In [9]:
import csv
import spacy   # NLTK has many different lematizers and POS taggers, 
## but they're not neural network powered and perforamnces would not differ much.
## Out of reason for speed, here we use spaCy as the POS tagger. Since this is medical texts, I expect the performance 
## of spaCy tagger might be worse more or less because of the training courpus is not medical.

DATA = './data/docs_50_complete_share'

In [3]:
spacy.__version__

'2.0.12'

### Get  Abstracts Only

Demo code below: (already be commented out)

In [ ]:
# with open(DATA) as csvfile:
#     papers = csv.reader(csvfile)
#     for i, row enumerate(papers):
#         if i == 0: continue
#         print(row)   # 7 cloumns as expected
#         if i == 10: break

In [21]:
abstracts = []
with open(DATA, encoding='utf-8') as csvfile:
    papers = csv.reader(csvfile)
    
    for i, row in enumerate(papers):
        if i == 0: continue
        abstracts.append(row[2])

print('Total %i abstracts.'%i)

Total 100 abstracts.


In [13]:
row

['27773734',
 'Liposomal prednisolone inhibits tumor growth in a spontaneous mouse mammary carcinoma model.',
 'Cancers are abundantly infiltrated by inflammatory cells that are modulated by tumor cells to secrete mediators fostering tumor cell survival and proliferation. Therefore, agents that interfere with inflammatory signaling molecules or specific immune cell populations have been investigated as anticancer drugs. Corticosteroids are highly potent anti-inflammatory drugs, whose activity is intensified when targeted by nanocarrier systems. Liposome-targeted corticosteroids have been shown to inhibit tumor growth in different syngeneic murine tumor models as well as human xenograft mouse models, which is attributed to a switch in the tumor microenvironment from a pro-inflammatory to an anti-inflammatory state. Despite the recognized value of implantation tumor models in preclinical research, the "acute" inflammation induced by inoculation of tumor cells together with the exponentia

### POS Parsing & NER

In this section, we will have every abstract parsed and thereby get their part of speech tag and the name entities as well as the name entity type tags for every word.Then, we will keep only the words having noun as part of speech tag and also theses words should not be identified as time, location, organization, person name, etc. the name words/phrases that apparently are not medical durg names or cancer names.

There remained nouns will be presented to the annotators along with tip for longer words (empirically, in this report, we'll define a long words with a leter limit of 9) if any. But the longer nouns are just head notes, the annotators will be told beforehand that they should decide with their discretion.

In [22]:
nlp = spacy.load("en_core_web_sm")

Demo code below: (already be commented out)

In [48]:
doc = nlp("Retinoids can block cell proliferation and induce apoptosis in tumor cells. The antitumoral effect of synthetic retinoids like Adapalene (ADA) on hepatoma cells (HepG2, Hep1B) was investigated. ADA at 10(-4)M efficiently induced apoptosis, reaching 61.7% in HepG2 and 79.1% in Hep1B after 72 h incubation. This was accompanied by up-regulation of pro-apoptotic bax and caspase 3, while bcl-2 was down-regulated, shifting the bax/bcl-2 ratio to >2.3 in hepatoma cells. ADA inhibits hepatoma cell growth in vitro and is a powerful inducer of hepatoma cell apoptosis.")

## Filteru the name entity type
ents = [(e.text, e.label_) for e in doc.ents]

filtered_abst = []
long_w = []
for token in doc:
    ## Exclude gene, having number in name
    if not token.text.isalpha():
        filtered_abst.append(token.text)
    ## Ignore time, location, person names, etc.
    elif token.tag_.startswith('NN'):
        if token.ent_type_ in ['PRODUCT', '']:
            filtered_abst.append('*[%s]*'%token.text)
        else: filtered_abst.append(token.text)
    else:
        filtered_abst.append(token.text)

    if token.text.__len__() > 9:
        long_w.append(token.text)
    

print(' '.join(filtered_abst))
print('\nYou might want to check these LONG word(s):')
print(' '.join(long_w))

a = input("\nType in words you'd like to annotate, seperate them with whitespace please:").strip()

*[Retinoids]* can block *[cell]* *[proliferation]* and induce *[apoptosis]* in *[tumor]* *[cells]* . The antitumoral *[effect]* of synthetic *[retinoids]* like Adapalene ( ADA ) on *[hepatoma]* *[cells]* ( HepG2 , Hep1B ) was investigated . *[ADA]* at 10(-4)M efficiently induced *[apoptosis]* , reaching 61.7 % in HepG2 and 79.1 % in Hep1B after 72 *[h]* *[incubation]* . This was accompanied by up - *[regulation]* of pro - apoptotic *[bax]* and *[caspase]* 3 , while bcl-2 was down - regulated , shifting the *[bax]* / bcl-2 *[ratio]* to > 2.3 in *[hepatoma]* *[cells]* . *[ADA]* inhibits *[hepatoma]* *[cell]* *[growth]* in vitro and is a powerful *[inducer]* of *[hepatoma]* *[cell]* *[apoptosis]* .

You might want to check these LONG word(s):
proliferation antitumoral investigated efficiently incubation accompanied regulation

Type in words you'd like to annotate, seperate them with whitespace please:retinoids apoptosis hepatoma


Below is the code for asking for annotation decisions row by row:

In [ ]:
identified_words_per_abst = []
for abst in abstracts:
    doc = nlp(abst)    
    ## Filteru the name entity type
    ents = [(e.text, e.label_) for e in doc.ents]
    
    filtered_abst = []
    long_w = []
    for token in doc:
        ## Exclude gene, having number in name
        if not token.text.isalpha():
            filtered_abst.append(token.text)
        ## Ignore time, location, person names, etc.
        elif token.tag_.startswith('NN'):
            if token.ent_type_ in ['PRODUCT', '']:
                filtered_abst.append('*[%s]*'%token.text)
            else: filtered_abst.append(token.text)
        else:
            filtered_abst.append(token.text)
        
        if token.text.__len__() > 9:
            long_w.append(token.text)
    
    ## Here goes the interactive annotation part:
    print(' '.join(filtered_abst))
    print('\nYou might want to check these LONG word(s):')
    print(' '.join(long_w))
    identified_words_per_abst.append(input("\nType in words you'd like to annotate, seperate them with whitespace please:").strip())

## Generate Automatic annotations

Since we don't have the chance to cross-annotation and cross-evaluate our annotation method with other team. We would like to treat the words:

1. is noun but not part of any name entity
OR
2. is noun as well as a 'PRODUCT' type of name entity ranther than location, country, time, etc.

together as our annotations.

###  Abstract Relevancy Classification

[Unfinished]

Considering the word length of scientific paper (usually 200~250 words) and many in-domain terms,the nowadays state-of-the-art -- neural language model trained on large and various open corpora and with specifically designed comprehension tasks, would be the definite off-shelf winner classfier for this task. Pretrained neural langauge is also especiallt suitable and useful for the data-scarse text mining problem.

What's better, if we can fine-tune a classifier head from scratch on the top of that neural language model,based on the recent researches, we expect the relevancy classification would be higher.Becuase the top classifier head would learn specifically about the langauge idiosyncracies of the scientific medical written language by research reuslts. 

Generate tsv annotation file (output).

## Annotation Method Evaluation inside Our Team

We deicide to adversarially evaluate and review our anootation tool within our team due to the time reason.

Task 3 require us to **progmatically** compare our consensused against the ground truth.

**Response**

[Unfinished.]

-----------------------------------------------------

**Proposed Improvement**

Due to we are running out of hackthon time, we'd like to give our hypethesis about the possible inaccuracy source/reason:
* Since even we've used the most cut-the-edge sequence labeling parsers, powered by powerful neural network and tons of corpus data, yet the medical research papers are quite biased and assumbly contained lots of OOV (out of vocabulary compared to the daily vocabulary and online vocabulary) recognized by the neural parsers. These OOV inavoidably will affect the recignition accuracy.

* A promising method would be fine tune the sequence label tagging layer (head) with a parameter-frozen backbone neural language model, which is massively trained and publicly shared by big company e.g. BERT of Google, GPT. Lnaguage model trained on large corpus learns the subtlenuances of human langauge thus would be expeted to be more roubust to different domains of natrual language.

* Empirically, I will also expect there will be some false negative medical terms such as 'Adapalene' shown above. But without bigger medical corpus, it would not be easy to get pass with these meical terms and jargons.This is also why the clinical and medical text ming so challenging: biased yet prevalent data scarcity.